In [1]:

from importlib import reload
import os
import pandas as pd
from io import StringIO
import itertools
import ipdb
import datetime
from collections import Counter

import h5py
import json
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

print(tf.__version__)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler

import mytf.s3utils as msu
import mytf.utils as mu
import mytf.plot as mp

1.14.0


Using TensorFlow backend.


In [2]:
tf.enable_eager_execution()


In [3]:
def save_loss(loss_history, wheredir):
    ts = mu.quickts()
    with open(f'{wheredir}/{ts}.json', 'w') as fd:
        json.dump(loss_history, fd)

In [57]:
# continue from 2019-12-01 notebook where I had a MemoryError, 

# 
# Grab and randomize since it's not huge...
vecs = [mu.read_h5_two(source_location='data/2019-12-07-train-balanced.h5', 
                    Xdataset=f'X_{i}',
                    Ydataset=f'Ylabels_{i}')
                 for i in [0, 1, 2, 3]]

X_train = np.concatenate([x[0] for x in vecs])
Ylabels_train = np.concatenate([x[1] for x in vecs])

# shuffle...
size = X_train.shape[0]
print(size)
indices = np.random.choice(range(size), size=size, replace=False)
X_train_shfl = X_train[indices]
Ylabels_train_shfl = Ylabels_train[indices].astype('int64')

34205


In [58]:
X_train.shape, Ylabels_train.shape, X_train_shfl.shape, Ylabels_train_shfl.shape

((34205, 64, 8), (34205,), (34205, 64, 8), (34205,))

In [59]:
Counter(Ylabels_train), Counter(Ylabels_train_shfl)

(Counter({0.0: 9455, 1.0: 7625, 2.0: 8286, 3.0: 8839}),
 Counter({2: 8286, 3: 8839, 0: 9455, 1: 7625}))

In [31]:
i = 0
with h5py.File('data/train.h5', 'r+') as fd:
#    print(list(fd.keys()))


#    X = fd[f'X_{i}'].__array__()
    Y = fd['dataset_0_Y'].__array__()
class_counts = tf.reduce_sum(Y, axis=0)
labels = np.argmax(Y, axis=1)

print(class_counts)
print(dict(Counter(labels)))
adict = dict(Counter(labels))
print([adict.get(i, 0) for i in [0, 1, 2, 3]])
#print()

tf.Tensor([1650.    0. 8286.    0.], shape=(4,), dtype=float64)
{0: 1650, 2: 8286}
[1650, 0, 8286, 0]


In [76]:
reload(mu)

<module 'mytf.utils' from '/home/ec2-user/SageMaker/aviation-pilot-physiology-hmm/mytf/utils.py'>

In [67]:
m = mu.load_model('history/2019-12-08T215137Z/00000_model.h5')

W1208 21:54:40.448859 140283925907264 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [75]:
m(tf.convert_to_tensor(X, dtype=tf.float32)[:3])
#type(tf.convert_to_tensor(X)[0][0][0])

<tf.Tensor: id=75755, shape=(3, 4), dtype=float32, numpy=
array([[ 0.27179655, -0.28642282,  0.22414565,  0.45910197],
       [-0.17425132, -0.03353608,  0.07828245,  0.05538325],
       [ 0.22939533, -0.1261261 ,  0.32558665,  0.4667279 ]],
      dtype=float32)>

In [83]:
for batch, blah in tqdm(enumerate(dataset_batches.take(size))):
    pass

1069it [00:00, 4570.92it/s]


In [84]:
size//32

1068

In [85]:
%%time 
loss_history = []
validation_loss_history = []
save_dir = 'history'
BATCH_SIZE = 32
ts = mu.quickts(); print('starting,', ts)
X = X_train_shfl
Ylabels = Ylabels_train_shfl
size = X_train_shfl.shape[0]
workdir = f'{save_dir}/{ts}'
os.mkdir(workdir)
#Counter(Ylabels_train), Counter(Ylabels_train_shfl)

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64,   dropout=0.2, recurrent_dropout=0.2,
                batch_input_shape=(None, 64, 8), 
              ),
    # 4 because 'A', 'B', 'C', 'D'.
    tf.keras.layers.Dense(4)])

class_weights = {0: 1., 1: 1., 2: 1., 3: 1.}
# parts = get_partitions(range(size), slice_size=BATCH_SIZE)
# training_indices = np.arange(0, X.shape[0], 1)
dataset_batches = mu.build_dataset_weighty_v3(
        {'x_train': X,
         'ylabels_train': Ylabels},  # 'ylabels_train'  if i have labels
        list(range(size)), 
        class_weights,
        batch_size=BATCH_SIZE)
    
with ipdb.launch_ipdb_on_exception():
    mu.do_train(
        model,
        dataset_batches,
        k=size,
        saveloc=workdir)
#    loss_history_this = [float(x) for x in loss_history_this]
#    loss_history.extend(loss_history_this)
#    #save_loss(loss_history, save_dir)

starting, 2019-12-08T220612Z
num slices 1068
size_remainder,  29


/home/ec2-user/SageMaker/aviation-pilot-physiology-hmm/mytf/utils.py:185: RuntimeWarning: divide by zero encountered in true_divide
  )/class_counts
1069it [07:28,  2.39it/s]

CPU times: user 7min 24s, sys: 761 ms, total: 7min 25s
Wall time: 7min 28s
